In [2]:
import scipy
from scipy.spatial.distance import squareform
import cellgrid
from initialization import init_uniform
import numpy as np
from MDAnalysis.lib.distances import self_distance_array
import itertools

In [2]:
%load_ext memory_profiler
%matplotlib inline

In [3]:
def _check_box(box):
    if box.shape == (6,):
        box1 = box[:3]
    else:
        box1 = box
    return box1

def pair_contact(cg,maxdist):
    indx,dist = cellgrid.capped_self_distance_array(cg._coordinates,maxdist,cg._box)
    mask = np.where(dist<maxdist)
    return indx[mask],dist[mask]

In [4]:
def bf_select(box, points, maxdist):
    distance = self_distance_array(points,box)
    distance = scipy.spatial.distance.squareform(distance)
    distance[np.tril_indices(distance.shape[0])] += (1.1*maxdist)
    mask = np.where((distance < maxdist))
    out = (np.column_stack((mask[0],mask[1])),distance[mask])
    return out

In [5]:
def cg_select(box,cellsize,points,cutoff_distance):
    box = _check_box(box)
    cg1 = cellgrid.CellGrid(box,cellsize,points)
    out_cg = pair_contact(cg1,cutoff_distance)
    return out_cg

# Cellsize > Cutoff_Distance

In [6]:
##Setting
box = np.array([100,100,100,90.,90.,90.],dtype=np.float32)
cellsize = 10.
cutoff_distance = 5.
Npoints = 1000

In [7]:
#Initialization
points = init_uniform(box,Npoints)

In [8]:
out_cg = cg_select(box,cellsize,points,cutoff_distance)

In [9]:
out_bf = bf_select(box,points,cutoff_distance)

# Testing

In [39]:
# Length
np.testing.assert_equal(out_bf[1].shape,out_cg[1].shape)
# Distances
np.testing.assert_array_almost_equal(np.sort(out_bf[1].astype(np.float64)),np.sort(out_cg[1]),decimal=5) #Comparing array values
# Rows,Column
np.testing.assert_array_equal(np.sort(out_cg[0][...,1]),np.sort(out_bf[0][...,1]))
np.testing.assert_array_equal(np.sort(out_cg[0][...,0]),np.sort(out_bf[0][...,0]))

# Variation of cellsize with Npoints for same cutoff distance

In [47]:
#Setting
cellsize = np.linspace(5,50,num=10,dtype=np.int32)
Npoints = np.logspace(2,5,num=10,dtype=np.int32)
cutoff = 5.
box = np.array([100,100,100,90.,90.,90.],dtype=np.float32)

In [ ]:
data = []
for npt in Npoints:
    points = init_uniform(box,npt)
    for cs in cellsize:
        time_build = %timeit -q -o -n 2 -r 3 cellgrid.CellGrid(_check_box(box),cs,points)
        time = %timeit -q -o -n 2 -r 3 cg_select(box,cs,points,cutoff_distance)
        print("Npoints {} CellSize {} Time {}".format(npt,cs,time.average))
        data.append((npt,cs,time.average,time_build.average))

Npoints 100 CellSize 5 Time 0.9880923333333461
Npoints 100 CellSize 10 Time 0.9785046666665949
Npoints 100 CellSize 15 Time 0.96560799999952
Npoints 100 CellSize 20 Time 0.9370001666666212
Npoints 100 CellSize 25 Time 0.9389494999998836
Npoints 100 CellSize 30 Time 0.9533588333333682
Npoints 100 CellSize 35 Time 0.9378786666663169
Npoints 100 CellSize 40 Time 0.9395280000001852
Npoints 100 CellSize 45 Time 0.9302074999998998
Npoints 100 CellSize 50 Time 1.061085833333588
Npoints 215 CellSize 5 Time 1.1700289999998859
Npoints 215 CellSize 10 Time 1.083472166666373
Npoints 215 CellSize 15 Time 1.000367166666668
Npoints 215 CellSize 20 Time 0.9751653333332797
Npoints 215 CellSize 25 Time 0.9728316666669343
Npoints 215 CellSize 30 Time 1.0134358333334603
Npoints 215 CellSize 35 Time 0.982873666666516
Npoints 215 CellSize 40 Time 0.9535659999998339
Npoints 215 CellSize 45 Time 0.9543596666665811
Npoints 215 CellSize 50 Time 0.9547366666668798
Npoints 464 CellSize 5 Time 1.100958833333607
Np

In [37]:
a = []
for item in itertools.combinations_with_replacement([0,-1,1],3):
    a.append(item)